In [1]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()
import os
import tqdm as notebook_tqdm
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

c:\Users\jatin\Documents\AI\virtual environments\venv_ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


In [4]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [5]:
DOCUMENT1 = "Big Basin Hike. big basin state park. ocean view summit. meteor trail. sleeping forest trail. middle ridge trail. People always told me that their favorite hike is in Big Basin, and now I know why!❤️. Lush green!🌲#BigBasinStatePark #HikeCalifornia #TrailLife #ExploreBigBasin #NatureLovers #RedwoodForest #CaliforniaHiking #DiscoverNature #OutdoorAdventure #SantaCruzMountains #HikingCulture #NatureEscape #HikingViews #WanderCalifornia"
DOCUMENT2 = 'Springtime in California means endless blooming hikes! 🌸🌿 The vibrant greens and beautiful views elevate my mood every time. Where should I explore next?#CaliforniaSpring #SpringHikes #HikingCalifornia #NatureLovers #BloomingSeason #ExploreCalifornia #OutdoorAdventure #ScenicTrails #NaturePhotography #GoOutside #vargasplataeu'
DOCUMENT3 = "Zion National Park,Utah.8C, some serious sun burns, a giant human poop (after all it is a scary hike) and what not. Angels Landing!🌌P.S. Angels landed on a very hot day! 😂Also sediments (not sentiments) are my new favourite! 🌚🌝"
DOCUMENT4 = "Rainbow Mt —> Plaza de Armas —> Sacred Valley 🦙🧡#RainbowMountain #Vinicunca #MontanaDeSieteColores #PeruAdventure #IncaTrail #HikingPeru #SacredValley #ValleSagrado #PeruTravel #CuscoRegion #AndeanCulture #IncaHeritage"

documents2 = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4]

In [6]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [7]:
import chromadb

DB_NAME = "googlecardb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

In [8]:
chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)
db.add(documents=documents2, ids=[str(i) for i in range(len(documents2))])

: 

In [ ]:
db.count()